<a href="https://colab.research.google.com/github/NMPau/Project/blob/main/PDA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.features import DivIcon
import warnings
import scipy as sp
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import seaborn as sns
sns.set_style('white')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
cd drive/My\ Drive

/content/drive/My Drive


In [4]:
cd Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [5]:
cd DA

/content/drive/My Drive/Colab Notebooks/DA


In [ ]:
!ls

airlines.csv  flights.csv    L_AIRPORT_ID.csv
airports.csv  L_AIRPORT.csv  PDA.ipynb


In [6]:
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')
flights = pd.read_csv('flights.csv', low_memory = False)
aircode1 = pd.read_csv('L_AIRPORT.csv')
aircode2 = pd.read_csv('L_AIRPORT_ID.csv')

Проверка дубликатов

In [ ]:
print(f'Duplicates in df: {flights.iloc[:,1:].duplicated().sum()}')

Duplicates in df: 0


Форматирование кодов аэропортов

In [7]:
aircode1 = aircode1.reset_index()
aircode2 = aircode2.reset_index()
aircodes = pd.merge(aircode1,aircode2,on='Description')
aircode_dict = dict(zip(aircodes['Code_y'].astype(str),aircodes['Code_x']))

In [8]:
flights['ORIGIN_AIRPORT'] = flights['ORIGIN_AIRPORT'].values.astype(str)
flights['DESTINATION_AIRPORT'] = flights['DESTINATION_AIRPORT'].values.astype(str)

for i in range(len(flights)):
    if len(flights['ORIGIN_AIRPORT'][i]) != 3:
        to_replace = flights['ORIGIN_AIRPORT'][i]
        value = aircode_dict[flights['ORIGIN_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)
for i in range(len(flights)):
    if len(flights['DESTINATION_AIRPORT'][i]) != 3:
        to_replace = flights['DESTINATION_AIRPORT'][i]
        value = aircode_dict[flights['DESTINATION_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)

изменение формата даты

In [7]:
month_dict={
    1:  'January',
    2:  'February',
    3:  'March',
    4:  'April',
    5:  'May',
    6:  'June',
    7:  'July',
    8:  'August',
    9:  'September',
    10: 'October',
    11: 'November',
    12: 'December'
}
flights['MONTH_desc'] = flights['MONTH'].apply(lambda m: month_dict[m])

In [8]:
day_of_week_dict = {
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday',
    7: 'Monday'
}
flights['DOW_desc'] = flights['DAY_OF_WEEK'].apply(lambda d: day_of_week_dict[d])

In [12]:
flights['DATE'] = pd.to_datetime(flights[['YEAR','MONTH','DAY']], yearfirst=True)

формат времени

In [13]:
import datetime
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure

In [14]:
def combine_date_heure(x):
    if pd.isnull(x[0]) or pd.isnull(x[1]):
        return np.nan
    else:
        return datetime.datetime.combine(x[0],x[1])
def create_flight_time(df, col):    
    liste = []
    for index, cols in df[['DATE', col]].iterrows():    
        if pd.isnull(cols[1]):
            liste.append(np.nan)
        elif float(cols[1]) == 2400:
            cols[0] += datetime.timedelta(days=1)
            cols[1] = datetime.time(0,0)
            liste.append(combine_date_heure(cols))
        else:
            cols[1] = format_heure(cols[1])
            liste.append(combine_date_heure(cols))
    return pd.Series(liste)

In [15]:
flights['SCHEDULED_DEPARTURE'] = create_flight_time(flights, 'SCHEDULED_DEPARTURE')

In [16]:
flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].apply(format_heure)
flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].apply(format_heure)
flights['ARRIVAL_TIME'] = flights['ARRIVAL_TIME'].apply(format_heure)
flights['WHEELS_OFF'] = flights['WHEELS_OFF'].apply(format_heure)
flights['WHEELS_ON'] = flights['WHEELS_ON'].apply(format_heure)

In [17]:
def hour(x):
  dt = x
  time = dt.time()
  hour = time.hour
  return hour

In [18]:
flights['DEPARTURE_HOUR'] = flights['SCHEDULED_DEPARTURE'].apply(hour)

In [8]:
#flights['SCHEDULED_DEPARTURE'] = np.ceil(flights['SCHEDULED_DEPARTURE']/600).apply(int)

In [19]:
flights

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE,DEPARTURE_HOUR
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,2015-01-01 00:05:00,23:54:00,-11.0,21.0,00:15:00,205.0,194.0,169.0,1448,04:04:00,4.0,04:30:00,04:08:00,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,2015-01-01 00:10:00,00:02:00,-8.0,12.0,00:14:00,280.0,279.0,263.0,2330,07:37:00,4.0,07:50:00,07:41:00,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,0
2,2015,1,1,4,US,840,N171US,SFO,CLT,2015-01-01 00:20:00,00:18:00,-2.0,16.0,00:34:00,286.0,293.0,266.0,2296,08:00:00,11.0,08:06:00,08:11:00,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,2015-01-01 00:20:00,00:15:00,-5.0,15.0,00:30:00,285.0,281.0,258.0,2342,07:48:00,8.0,08:05:00,07:56:00,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,0
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,2015-01-01 00:25:00,00:24:00,-1.0,11.0,00:35:00,235.0,215.0,199.0,1448,02:54:00,5.0,03:20:00,02:59:00,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,2015,12,31,4,B6,688,N657JB,LAX,BOS,2015-12-31 23:59:00,23:55:00,-4.0,22.0,00:17:00,320.0,298.0,272.0,2611,07:49:00,4.0,08:19:00,07:53:00,-26.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-31,23
5819075,2015,12,31,4,B6,745,N828JB,JFK,PSE,2015-12-31 23:59:00,23:55:00,-4.0,17.0,00:12:00,227.0,215.0,195.0,1617,04:27:00,3.0,04:46:00,04:30:00,-16.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-31,23
5819076,2015,12,31,4,B6,1503,N913JB,JFK,SJU,2015-12-31 23:59:00,23:50:00,-9.0,17.0,00:07:00,221.0,222.0,197.0,1598,04:24:00,8.0,04:40:00,04:32:00,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-31,23
5819077,2015,12,31,4,B6,333,N527JB,MCO,SJU,2015-12-31 23:59:00,23:53:00,-6.0,10.0,00:03:00,161.0,157.0,144.0,1189,03:27:00,3.0,03:40:00,03:30:00,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-31,23


In [20]:
corr = flights.corr(method='pearson')

Для построения модели для прогнозирования будем использовать опоздание по колонке ARRIVAL_DELAY

In [21]:
corr['ARRIVAL_DELAY']

YEAR                        NaN
MONTH                 -0.036793
DAY                   -0.003097
DAY_OF_WEEK           -0.017027
FLIGHT_NUMBER          0.018419
DEPARTURE_DELAY        0.944672
TAXI_OUT               0.227319
SCHEDULED_TIME        -0.030029
ELAPSED_TIME           0.029120
AIR_TIME              -0.007108
DISTANCE              -0.025444
TAXI_IN                0.116628
ARRIVAL_DELAY          1.000000
DIVERTED                    NaN
CANCELLED                   NaN
AIR_SYSTEM_DELAY       0.247187
SECURITY_DELAY         0.009655
AIRLINE_DELAY          0.609351
LATE_AIRCRAFT_DELAY    0.522013
WEATHER_DELAY          0.264799
DEPARTURE_HOUR         0.100066
Name: ARRIVAL_DELAY, dtype: float64

In [ ]:
flights.columns

Убираем ненужные колонки

In [22]:
flights = flights.drop(['YEAR', 'FLIGHT_NUMBER','DIVERTED','CANCELLED', 'TAIL_NUMBER', 'AIR_TIME', 'CANCELLATION_REASON'], axis = 1)

In [23]:
flights = flights.drop(['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'], axis = 1)

Проверяем, есть ли данные с пропущенными значениями

In [16]:
missing_df = flights.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(flights.shape[0]-missing_df['missing values'])/flights.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,ARRIVAL_DELAY,105071,98.194371
1,ELAPSED_TIME,105071,98.194371
2,TAXI_IN,92513,98.410178
3,WHEELS_ON,92513,98.410178
4,ARRIVAL_TIME,92513,98.410178
5,TAXI_OUT,89047,98.469741
6,WHEELS_OFF,89047,98.469741
7,DEPARTURE_TIME,86153,98.519474
8,DEPARTURE_DELAY,86153,98.519474
9,SCHEDULED_TIME,6,99.999897


Удаляем строки с пропущенными значениями (для нужных переменных уровень заполненности начинаеется с 98% это хорошо - не будем заполнять средним значением). По переменным с причинами задержки - пропуски заполним нулевым значением (будем считать, что была иная причина задержки)

Удаляем пустые значения

In [24]:
flights.dropna(subset=['ARRIVAL_DELAY','ELAPSED_TIME','WHEELS_ON','TAXI_IN','ARRIVAL_TIME','WHEELS_OFF','TAXI_OUT','DEPARTURE_DELAY','DEPARTURE_TIME'],inplace=True)

In [25]:
missing_df = flights.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(flights.shape[0]-missing_df['missing values'])/flights.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,MONTH,0,100.0
1,ARRIVAL_DELAY,0,100.0
2,ARRIVAL_TIME,0,100.0
3,SCHEDULED_ARRIVAL,0,100.0
4,TAXI_IN,0,100.0
5,WHEELS_ON,0,100.0
6,DISTANCE,0,100.0
7,ELAPSED_TIME,0,100.0
8,SCHEDULED_TIME,0,100.0
9,DATE,0,100.0


Заполняем пропущенные значения нулем

In [ ]:
flights = flights.fillna(0)

Найти аэропорт с минимальной задержкой вылета

In [ ]:
a0 = flights.loc[flights['DEPARTURE_DELAY'] == flights['DEPARTURE_DELAY'].min()][['ORIGIN_AIRPORT']]

In [ ]:
AIRPORT_min_delay = airports.loc[airports['IATA_CODE'] == a0['ORIGIN_AIRPORT'].values[0]][['AIRPORT']]

In [ ]:
AIRPORT_min_delay

,AIRPORT
9,Adak Airport


Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport

In [ ]:
LA = airports.loc[airports['AIRPORT'] == 'Los Angeles International Airport'][['IATA_CODE']]

In [ ]:
b = flights.loc[flights['DESTINATION_AIRPORT'] == LA['IATA_CODE'].values[0]][['AIRLINE', 'ARRIVAL_DELAY']]

In [ ]:
b1 = b.groupby(['AIRLINE']).mean()
b1

,ARRIVAL_DELAY
AIRLINE,
AA,2.122856
AS,-0.525700
B6,-0.022152
DL,1.398882
F9,10.288235
HA,-4.653317
MQ,18.302469
NK,17.138697
OO,8.202029


Под самой пунктуальной будем понимать авиакомпанию, чье среднее значение самое близкое к нулю (то есть время минимально отклоняется от графика)

In [ ]:
myNumber = 0
m = min(b1['ARRIVAL_DELAY'].values, key=lambda x:abs(x-myNumber))

In [ ]:
ar = b1.loc[b1['ARRIVAL_DELAY'] == m]

In [ ]:
PUNCTUAL_AIRLINE = airlines.loc[airlines['IATA_CODE'] == ar.index[0]][['AIRLINE']]

In [ ]:
PUNCTUAL_AIRLINE

,AIRLINE
4,JetBlue Airways


Найти аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение)

In [ ]:
TO = flights[['ORIGIN_AIRPORT', 'TAXI_OUT']]
TI = flights[['DESTINATION_AIRPORT', 'TAXI_IN']]
to = TO.groupby(['ORIGIN_AIRPORT']).mean()
ti = TI.groupby(['DESTINATION_AIRPORT']).mean()

In [ ]:
taxi = to.assign(TAXI_IN = ti.TAXI_IN)
taxi = taxi.assign(TAXI_ALL = taxi.TAXI_OUT + taxi.TAXI_IN)

In [ ]:
taxi

,TAXI_OUT,TAXI_IN,TAXI_ALL
ORIGIN_AIRPORT,,,
ABE,13.325223,3.968635,17.293858
ABI,9.182418,4.003309,13.185727
ABQ,11.948608,5.573155,17.521763
ABR,16.502046,4.679945,21.181991
ABY,12.246849,3.429022,15.675871
...,...,...,...
WRG,7.759547,3.704067,11.463615
WYS,11.903846,4.666667,16.570513
XNA,14.605457,5.102742,19.708200


In [ ]:
max_TAXi = taxi.loc[taxi['TAXI_ALL'] == taxi['TAXI_ALL'].max()]

In [ ]:
max_TAXi_AIRPORT = airports.loc[airports['IATA_CODE'] == max_TAXi.index[0]][['AIRPORT']]

In [ ]:
max_TAXi_AIRPORT

,AIRPORT
182,LaGuardia Airport (Marine Air Terminal)


Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета

In [ ]:
!pip install -U lightautoml

In [27]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.metrics import f1_score, log_loss, roc_auc_score, roc_curve, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import train_test_split
import time
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.ml_algo.tuning.optuna import OptunaTuner
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco

In [28]:
N_THREADS = 4 # threads cnt for lgbm and linear models
N_FOLDS = 3 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 300 # Time in seconds for automl run
TARGET_NAME = 'ARRIVAL_DELAY' # Target column name

In [29]:
import torch
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [30]:
Origin_Airport = np.random.choice(flights['ORIGIN_AIRPORT'].unique()) 
data = flights[flights['ORIGIN_AIRPORT'] == Origin_Airport]

In [31]:
data

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DATE,DEPARTURE_HOUR
316,1,1,4,MQ,CVG,MIA,2015-01-01 06:00:00,05:56:00,-4.0,15.0,06:11:00,153.0,150.0,948,08:14:00,12.0,08:33:00,08:26:00,-7.0,2015-01-01,6
1056,1,1,4,DL,CVG,ATL,2015-01-01 07:10:00,07:08:00,-2.0,36.0,07:44:00,102.0,101.0,373,08:46:00,3.0,08:52:00,08:49:00,-3.0,2015-01-01,7
1168,1,1,4,DL,CVG,TPA,2015-01-01 07:20:00,07:19:00,-1.0,9.0,07:28:00,141.0,116.0,773,09:13:00,2.0,09:41:00,09:15:00,-26.0,2015-01-01,7
1219,1,1,4,F9,CVG,FLL,2015-01-01 07:25:00,07:23:00,-2.0,10.0,07:33:00,150.0,143.0,932,09:39:00,7.0,09:55:00,09:46:00,-9.0,2015-01-01,7
1307,1,1,4,DL,CVG,RSW,2015-01-01 07:31:00,07:22:00,-9.0,27.0,07:49:00,153.0,150.0,879,09:46:00,6.0,10:04:00,09:52:00,-12.0,2015-01-01,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5816536,12,31,4,MQ,CVG,DFW,2015-12-31 17:27:00,17:18:00,-9.0,11.0,17:29:00,168.0,192.0,812,19:03:00,27.0,19:15:00,19:30:00,15.0,2015-12-31,17
5817044,12,31,4,EV,CVG,ORD,2015-12-31 18:01:00,17:58:00,-3.0,9.0,18:07:00,84.0,67.0,264,17:55:00,10.0,18:25:00,18:05:00,-20.0,2015-12-31,18
5817053,12,31,4,MQ,CVG,MIA,2015-12-31 18:03:00,19:36:00,93.0,9.0,19:45:00,154.0,142.0,948,21:52:00,6.0,20:37:00,21:58:00,81.0,2015-12-31,18
5817486,12,31,4,MQ,CVG,ORD,2015-12-31 18:45:00,18:37:00,-8.0,30.0,19:07:00,90.0,97.0,264,19:04:00,10.0,19:15:00,19:14:00,-1.0,2015-12-31,18


In [32]:
data = data.drop(['DEPARTURE_DELAY', 'ARRIVAL_TIME','WHEELS_ON','SCHEDULED_ARRIVAL', 'WHEELS_OFF', 'DEPARTURE_TIME','SCHEDULED_DEPARTURE'], axis =1)

In [33]:
data.columns

Index(['MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'ORIGIN_AIRPORT',
       'DESTINATION_AIRPORT', 'TAXI_OUT', 'SCHEDULED_TIME', 'ELAPSED_TIME',
       'DISTANCE', 'TAXI_IN', 'ARRIVAL_DELAY', 'DATE', 'DEPARTURE_HOUR'],
      dtype='object')

In [34]:
%%time

train_data, test_data = train_test_split(data, 
                                         test_size=TEST_SIZE,  
                                         random_state=RANDOM_STATE)
print('Data splitted. Parts sizes: train_data = {}, test_data = {}'
              .format(train_data.shape, test_data.shape))

Data splitted. Parts sizes: train_data = (17096, 14), test_data = (4274, 14)
CPU times: user 9.49 ms, sys: 2.98 ms, total: 12.5 ms
Wall time: 14.5 ms


In [35]:
task = Task('reg', ) 
            #loss = 'mae', metric = 'mae')

In [36]:
roles = {'target': TARGET_NAME}

In [37]:
automl = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
)

In [ ]:
%%time 
oof_pred = automl.fit_predict(train_data, roles = roles, verbose = 1)

In [39]:
print(automl.create_model_str_desc())

Final prediction for new objects (level 0) = 
	 0.68584 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.23175 * (3 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.08242 * (3 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) 


In [40]:
%%time

test_pred = automl.predict(test_data)
print(f'Prediction for te_data:\n{test_pred}\nShape = {test_pred.shape}')

Prediction for te_data:
array([[ 47.724495 ],
       [  7.8562174],
       [ -3.8423252],
       ...,
       [ -5.7148414],
       [ 13.03556  ],
       [-16.839918 ]], dtype=float32)
Shape = (4274, 1)
CPU times: user 975 ms, sys: 8.78 ms, total: 984 ms
Wall time: 683 ms


In [41]:
from sklearn.metrics import mean_absolute_error
from numpy import sqrt 
print(f'TRAIN out-of-fold score: {mean_absolute_error(train_data[TARGET_NAME].values, oof_pred.data[:, 0])}')
print(f'HOLDOUT score: {mean_absolute_error(test_data[TARGET_NAME].values, test_pred.data[:, 0])}')

TRAIN out-of-fold score: 18.841903656805663
HOLDOUT score: 18.944433894709178


In [42]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse_val = rmse(np.array(test_pred.data[:, 0]), np.array(test_data[TARGET_NAME].values))
print("rms error is: " + str(rmse_val))

rms error is: 39.844882467732184


In [43]:
from sklearn.metrics import r2_score
print(f'R^2 train: {r2_score(train_data[TARGET_NAME].values, oof_pred.data[:, 0])}')
print(f'R^2 test: {r2_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])}')

R^2 train: 0.14928790803841907
R^2 test: 0.1514305310666083


In [44]:
from sklearn.metrics import mean_squared_error
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)
print('Check scores:')
print('OOF score: {}'.format(sqrt(mean_squared_error(train_data[TARGET_NAME].values[not_nan], oof_pred.data[not_nan][:, 0]))))
print('TEST score: {}'.format(sqrt(mean_squared_error(test_data[TARGET_NAME].values,test_pred.data[:, 0]))))

Check scores:
OOF score: 40.5020788282103
TEST score: 39.844882467732184


In [45]:
td = test_data[['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'ARRIVAL_DELAY']]
df = pd.DataFrame(test_pred.data[:, 0], columns = ['Prediction_ARRIVAL_DELAY'])

In [46]:
td.index = np.arange(0,len(td))

In [47]:
a = pd.concat([td,df], sort=False, axis=1)
a.head()

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,ARRIVAL_DELAY,Prediction_ARRIVAL_DELAY
0,CVG,IAH,37.0,47.724495
1,CVG,ATL,-3.0,7.856217
2,CVG,XNA,-9.0,-3.842325
3,CVG,ATL,-8.0,-4.319790
4,CVG,ORD,4.0,20.911352


In [48]:
final = []

In [49]:
for uniq_aiport in a['DESTINATION_AIRPORT'].unique():
  rmse = np.sqrt(mean_squared_error(a.ARRIVAL_DELAY[a['DESTINATION_AIRPORT'] == uniq_aiport], a.Prediction_ARRIVAL_DELAY[a['DESTINATION_AIRPORT'] == uniq_aiport]))
  final.append([Origin_Airport, uniq_aiport, rmse])


In [50]:
F = pd.DataFrame(final)
F.columns = ['Arrival_Airport', 'Destination_Airport', 'RMSE']

In [51]:
F = F.sort_values(by=['RMSE'])

In [52]:
F.head(3)

,Arrival_Airport,Destination_Airport,RMSE
36,CVG,PIT,6.567139
33,CVG,MCI,11.509010
34,CVG,RDU,13.808995
